In [1]:
import torch
import torch.nn as nn
from torchvision.models import inception_v3
from torch.optim import SGD, Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import os

/Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <CAE66874-17C2-35C9-9C4D-6BA9770AF17F> /Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <459875AA-DE2C-366B-9C44-90D4B3887080> /Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
transform = transforms.Compose([
                                transforms.Resize([342], interpolation=transforms.InterpolationMode.BILINEAR),
                                transforms.CenterCrop([299,299]),
                                transforms.ToTensor(), # Rescaled to 0-1 aswell
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # Mean and std of imageNet
])

target_transform = transforms.Compose([
                                transforms.Resize([342], interpolation=transforms.InterpolationMode.BILINEAR),
                                transforms.CenterCrop([299,299]),
                                transforms.ToTensor(),
])

In [3]:
class CustomDataset(Dataset):
    def __init__(self, path, transform=None, target_transform=None):
        self.transform= transform
        self.target_transform = target_transform
        self.path = path
        self.count = 0
        for i in os.listdir(self.path)[1:]: # [1:] because MacOS has .DS_Store at 0th index
            for j in os.listdir(os.path.join(self.path,i)):
                self.count += 1
        
    def __len__(self):
        return self.count
    
    def __getitem__(self, idx):
        # Lazy loading
        
        directory_idx = idx//100
        file_idx = idx%100
        
        label = directory_idx
        
        directory = os.listdir(self.path)[directory_idx + 1]
        flower_directory = os.path.join(self.path, directory)
        image_path = os.path.join(flower_directory, os.listdir(flower_directory)[file_idx])
    
        image = Image.open(image_path)
        image = transform(image)

        return image, label
        

In [4]:
dataset = CustomDataset('tiny_FR', transform)

In [5]:
torch.manual_seed(10)
index = torch.randperm(500)

In [6]:
data_train = Subset(dataset,index[:400])
data_valid = Subset(dataset,index[400:])

In [7]:
torch.manual_seed(1)
data = DataLoader(data_train, 64, shuffle=True)
data_vl = DataLoader(data_valid, 64, shuffle=True)

In [8]:
next(iter(data))[0].shape

torch.Size([64, 3, 299, 299])

In [9]:
model = inception_v3(weights="IMAGENET1K_V1")

In [10]:
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [11]:
for param in model.parameters():
    param.requires_grad = False

In [12]:
num_ftrs = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(num_ftrs, 5)

num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 5)

In [13]:
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print(param.shape)
        print("\t",name)
        


torch.Size([5, 768])
	 AuxLogits.fc.weight
torch.Size([5])
	 AuxLogits.fc.bias
torch.Size([5, 2048])
	 fc.weight
torch.Size([5])
	 fc.bias


In [14]:
for i in model.fc.parameters():
    print(i.shape)

torch.Size([5, 2048])
torch.Size([5])


In [15]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [16]:
def train(epoch, data, data_vl):
    train_loss = [0] * epoch
    train_accuracy = [0] * epoch
    
    valid_loss = [0] * epoch
    valid_accuracy = [0] * epoch
    model.train()
    for i in range(epoch):
        for x_batch, y_batch in data:
            #print(x_batch.shape)
            
            pred, _ = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            train_loss[i] += loss.item() * x_batch.size(0)
            correct = torch.sum(torch.argmax(torch.softmax(pred,axis=1), axis=1) == y_batch)
            train_accuracy[i] += correct.to(torch.float32).item()
        train_loss[i] /= len(data.dataset)
        train_accuracy[i] /= len(data.dataset)
        
        with torch.no_grad():
            for x_batch, y_batch in data_vl:
                pred, _ = model(x_batch)
                loss = loss_fn(pred, y_batch)
                valid_loss[i] += loss.item() * x_batch.size(0)
                correct = torch.sum(torch.argmax(torch.softmax(pred,axis=1), axis=1) == y_batch)
                valid_accuracy[i] += correct.item()
            valid_loss[i] /= len(data_vl.dataset)
            valid_accuracy[i] /= len(data_vl.dataset)
        print(f'Epoch {i+1} accuracy: {train_accuracy[i]:.4f} val_accuracy:{valid_accuracy[i]:.4f}')
        print(f'Epoch {i+1} loss: {train_loss[i]:.4f} val_loss:{valid_loss[i]:.4f}')
        print()
            

In [17]:
train(50, data, data_vl)

Epoch 1 accuracy: 0.1825 val_accuracy:0.2000
Epoch 1 loss: 1.6545 val_loss:1.6005

Epoch 2 accuracy: 0.2825 val_accuracy:0.2900
Epoch 2 loss: 1.5752 val_loss:1.5482

Epoch 3 accuracy: 0.3625 val_accuracy:0.4300
Epoch 3 loss: 1.5045 val_loss:1.4560

Epoch 4 accuracy: 0.5300 val_accuracy:0.5600
Epoch 4 loss: 1.3750 val_loss:1.3321

Epoch 5 accuracy: 0.6550 val_accuracy:0.6100
Epoch 5 loss: 1.2833 val_loss:1.2632

Epoch 6 accuracy: 0.6975 val_accuracy:0.6300
Epoch 6 loss: 1.2005 val_loss:1.2451

Epoch 7 accuracy: 0.7375 val_accuracy:0.6700
Epoch 7 loss: 1.1072 val_loss:1.1583

Epoch 8 accuracy: 0.7600 val_accuracy:0.7200
Epoch 8 loss: 1.0651 val_loss:1.0972

Epoch 9 accuracy: 0.7850 val_accuracy:0.7400
Epoch 9 loss: 0.9963 val_loss:1.0282

Epoch 10 accuracy: 0.7975 val_accuracy:0.7300
Epoch 10 loss: 0.9489 val_loss:0.9789

Epoch 11 accuracy: 0.7825 val_accuracy:0.7200
Epoch 11 loss: 0.8924 val_loss:0.9721

Epoch 12 accuracy: 0.8225 val_accuracy:0.7600
Epoch 12 loss: 0.8646 val_loss:0.9293

In [18]:
from sklearn.metrics import f1_score, confusion_matrix